In [1]:
from random import randint
import os
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score , roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, Input
from tensorflow.keras.layers import Flatten, MaxPool2D, AvgPool2D
from tensorflow.keras.layers import BatchNormalization, Reshape, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model, load_model

from PIL import Image

seed = 123

In [2]:
# ao carregarmos cada um dos batches, pré processaremos e faremos 'data augmentation', através de:

# carregamento 
def get_images_n_labels(dataframe, path = 'treino_full_non_preproc/', 
                        series_name = 'img', label_name = 'pos', i = 300, j = 300):
    
    x = []
    y = []
    
    for n in range(dataframe.shape[0]):
        
        img = cv2.imread(path + dataframe[series_name].iloc[n])
        img = cv2.resize(img, (i, j))
        x.append(img)
        y.append(dataframe[label_name].iloc[n])
        
    return np.array(x), np.array(y)

# pré processamento

def segmentation(img):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    sure_bg = cv2.dilate(opening,kernel,iterations = 3)
    res = cv2.bitwise_and(img,img,mask = sure_bg)
    
    return res

def bgr_CLAHE(img):
    
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit = 2.0,tileGridSize = (6, 6))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    
    return img

def img_preproc(img):
    
    img = bgr_CLAHE(segmentation(img))
    img = img / 255

    
    return img

# augmentation

def zoom(img, original_dim = [300, 300], h_slice = [25, 275], v_slice = [25, 275]):

    img = img[v_slice[0] : v_slice[1], h_slice[0] : h_slice[1]]
    img = cv2.resize(img, (original_dim[0], original_dim[1]))
    
    return img 

# horizontal shift
def h_shift(image, original_dim = [300, 300], shift = 5):
    
    T_x = shift
    T_y = 0
    
    M = np.array([[1, 0, T_x], [0, 1, T_y]], dtype = 'float32')
    img_transladada = cv2.warpAffine(image, M, (original_dim[0], original_dim[1]))
    img = img_transladada[0 : original_dim[0], shift : original_dim[1]]
    img = cv2.resize(img, (original_dim[0], original_dim[1]))
    
    return img

# vertical shift
def v_shift(image, original_dim = [300, 300], shift = 5):
    
    T_x = 0
    T_y = shift
    
    M = np.array([[1, 0, T_x], [0, 1, T_y]], dtype = 'float32')
    img_transladada = cv2.warpAffine(image, M, (original_dim[0], original_dim[1]))
    img = img_transladada[shift : original_dim[0], 0 : original_dim[1]]
    img = cv2.resize(img, (original_dim[0], original_dim[1]))
    
    return img

def rotation_90(img):

    rows, cols, chnls = img.shape
    M = cv2.getRotationMatrix2D(((cols - 1) / 2.0, (rows - 1) / 2.0), 90, 1)
    img = cv2.warpAffine(img, M, (cols, rows))
    
    return img

def data_augmentation(x, y):
    
    augmentation_imgs = []
    augmentation_labels = []

    for n in range(len(x)):

        image = x[n]
        classe = y[n]
        augment = randint(0, 2) # booleano (33.3% de chance de aplicar augmentation)

        if augment == 1:
            process = randint(0, 5) # seleção aleatória do processo de augmentation

            if process == 0:
                image = cv2.flip(image, 0)
                augmentation_imgs.append(image) # horizontal flip
                augmentation_labels.append(classe)

            if process == 1:
                image = zoom(image)
                augmentation_imgs.append(image) # zoom 0.2
                augmentation_labels.append(classe)

            if process == 2:
                image = h_shift(image)
                augmentation_imgs.append(image) # horizontal shift
                augmentation_labels.append(classe)

            if process == 3:
                image = v_shift(image)
                augmentation_imgs.append(image) # vertical shift
                augmentation_labels.append(classe)

            if process == 4:
                image = rotation_90(image)
                augmentation_imgs.append(image) # rotaton 90°
                augmentation_labels.append(classe)
                
    return augmentation_imgs, augmentation_labels

# consolidando os processos acima em um função
def single_batch_prep(batch_df, width = 300, height = 300):
    
    x, y = get_images_n_labels(batch_df)

    # pré processamento de imagens
    x = [img_preproc(i) for i in x]

    # data augmentation
    augmentation_imgs, augmentation_labels = data_augmentation(x, y)
    
    return np.array(x + augmentation_imgs), np.array(list(y) + augmentation_labels)


def get_batch(nn_name, batch_desc, valid_desc, valid_dir, batch_size, epochs, 
              encode = False):
    
    model = load_model(nn_name)
    batch = pickle.load(open(batch_desc, 'rb'))
    x_treino, y_treino = single_batch_prep(batch)
    
    valid = pickle.load(open(valid_desc, 'rb'))
    x_valid, y_valid = get_images_n_labels(valid, path = valid_dir)
    
    if encode == True:
        model.fit(x_treino, x_treino, batch_size = batch_size, epochs = epochs, 
                  validation_data = (x_valid, x_valid))
                  
    model.save(nn_name)

In [3]:
px_h, px_v, chnls = 300, 300, 3
ipt_e = Input(shape = (px_h, px_v, chnls))

encoder = Conv2D(50, (3, 3), input_shape = (px_h, px_v, chnls), activation = 'relu', padding = 'same')(ipt_e)
encoder = MaxPool2D((5, 5))(encoder)
encoder = Conv2D(50, (5, 5), activation = 'relu', padding = 'same')(encoder)
encoder = MaxPool2D((5, 5))(encoder)
encoder = Conv2D(50, (5, 5), activation = 'relu', padding = 'same')(encoder)
encoder = Flatten()(encoder)

encoder = Dense(60, name = 'gargalo')(encoder)

decoder = Dense(7200)(encoder)
decoder = Reshape(target_shape = (12, 12,50))(decoder)
decoder = Conv2D(50, (3, 3), activation = 'relu', padding = 'same')(decoder)
decoder = UpSampling2D((5, 5))(decoder)
decoder = Conv2D(50, (5, 5), activation = 'relu', padding = 'same')(decoder)
decoder = UpSampling2D((5, 5))(decoder)
decoder = Conv2D(4, (3, 3), activation = 'relu', padding = 'same')(decoder)
decoder = Conv2D(3, (3, 3), activation = 'sigmoid', padding = 'same')(decoder)

autoencoder = Model(inputs = ipt_e, outputs = decoder, name = 'autoencoder')
autoencoder.compile(loss = 'mean_squared_error')

autoencoder.save('ae.h5') # <- atenção: não rodar esta linha várias vezes, rodar apenas ao criar o encoder,
                          #    rodar esta linha sobrescreverá o encoder já treinado

In [7]:
get_batch('ae.h5', 'batch_0.pkl', 'df_valid.pkl', 'isic_2017_validacao/', 25, 100, encode = True)

Epoch 1/100
25/25 [==============================] - 116s 5s/step - loss: 0.0588 - val_loss: 23534.2305
Epoch 2/100
25/25 [==============================] - 108s 4s/step - loss: 0.0348 - val_loss: 23538.6152
Epoch 3/100
18/25 [====================>.........] - ETA: 29s - loss: 0.0290

KeyboardInterrupt: 

In [6]:
get_batch('ae.h5', 'batch_1.pkl', 'df_valid.pkl', 'isic_2017_validacao/', 25, 100, encode = True)

In [43]:
get_batch('ae.h5', 'batch_2.pkl', 'df_valid.pkl', 'isic_2017_validacao/', 25, 100, encode = True)

In [44]:
get_batch('ae.h5', 'batch_3.pkl', 'df_valid.pkl', 'isic_2017_validacao/', 25, 100, encode = True)

In [45]:
get_batch('ae.h5', 'batch_4.pkl', 'df_valid.pkl', 'isic_2017_validacao/', 25, 100, encode = True)